> Attention Is All You Need

<a href="https://nlp.seas.harvard.edu/2018/04/03/attention.html#encoder-and-decoder-stacks" target="_blank">Harvard Notebook</a>

<a href="https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec">Towardsdatascience Notebook</a>

# Transorfmer

Transformer, a bit better: <a href="https://glassboxmedicine.com/2019/08/15/the-transformer-attention-is-all-you-need/"> article</a>.

Embedding weight matrix $W$ has shape: number of embeddings x embedding dimension.

number of embedding = `vocab_size`, each word has an embedding.

embedding dimension = `d_model`, it's your choice, in the paper it is 512.

In `pytorch` you use `nn.Embedding` function. You can use pre-trained embeddings (e.g. GloVe), or you can randomly initialize them and train them. `Transformer` learns its own word embeddings:


In [1]:
from typing import Type, List
import torch
import torch.nn as nn

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.embed = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    def forward(self, x):
        return self.embed(x)

## Positional Encoding
In transformer all the words of a sentence are processed simultaneously, so there is no inherent word ordering that you have on a RNN. The authors of the Transformer propose adding a "positional encoding" to address this problem. There were 2 options for Positional encoding vectors:
1. learning the positional encoding vectors (add trainable parameters).
2. calculating the positional encoding vectors using an equation (requires no trainable parameters).

Opted for option 2. Formulas:

$PE_{(pos, 2i)} = sin(pos / 10000^{2i/d_{model}})$

$PE_{(pos, 2i+i)} = cos(pos / 10000^{2i/d_{model}})$

where `pos` is the position of the word in the sentence, and `i` is the index into the embedding dimension. If model = `d_model=512`, then than `i` range from 0 to 512 / 2.

This means that an english sentence: "I like trees" will be converted to 3 vectors (one vector for each word): <img src='../img/transformer/wordembedding_positional.png' width="500" height="500">
We do exactely the same thing also for the correct output target sequence "Me gustas los arboles".


The shape of the input Tensors (and output Tensor) with embedding and positional encoder considered is: `n_batches`, `L` and `512`. `nbatches` is the batch size, `L` is the lenght of the sequence, in this case would be 3 for the input and output, and `512` is the emedding dimensions (number of columns representing the vectors column of the word).


## Encoder
The encoder has 6 identical layers. What goes in is an English sentence, e.g. "I like trees", represented in the "word embeddings + positional encodigns" format we just talked about. What comes out is a different representation of the sentence.

Each layer contains 2 sub-layers:
1. Multi-head attention
2. feed-forward network

<img src="../img/transformer/encoder.png" width="400" height="400">

In [2]:
class Encoder(nn.Module):
    """Encoder stacked with 6 layers."""
    
    def __init__(self, layer: Type[nn.Module], N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = layerNorm(layer.size)
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
class EncoderLayer(nn.Module):
    """Self-attention and feed forward."""
    def __init__(self, size: int = 512, self_attn: Type[nn.Module] = None,
                 feed_forward: Type[nn.Module] = None, dropout = 0.1):
        super(EncoderLayer, self).__init__()
        

`EncoderLayer` has `size = d_model`, columns of the embeddings. `size = d_model = 512`.

### Multi-head attention
`x`, as embedding + position_encoder, is taken up by the first `EncoderLayer`. After, the others 5 layers will pick up a different representation of `x`, which came out from the previous `EncoderLayer`.

$$Attention(Q,K,V) = softmax( \frac{QK^T}{ \sqrt{dk}})  V$$

In the attention mechanism we split the `d_model` into `heads`. Originally `heads=8`. This means that the dimensionality of the `Tensor` is different compare the `size` they come in.

$Q = Tensor(n_{batches}, 8, L_q, 64)$

$K = Tensor(n_{batches}, 8, L_k, 64)$

$V = Tensor(n_{batches}, 8, L_v, 64)$

$Q$, $K$, $V$ do have different weights. They initially have been randomly defined (or better using Xavier Initialization). 

_Note_

_You need to think to the Q, K, V dimensions as if you have a `batch` of sentences, these sentences will be needed to further split into 8 `heads`, these heads have the lenght of the sentence but with `d_model / 8 = 64` columns._

`mask` is a `Tensor` but I don't know the size. `unsqueeze` add a third dimension, for example `unsqueeze(1)` add a 3rd dimension to the second position e.g. [1,4], [1,1,4].

What enter the `MultiHeadedAttention` need also to come out in the same `shape`. `shape = n_batches, L, 512`.

In the 8 `heads` we did a different matrix multiplication for each of the eight heads, this is what is meant by _multi-headed_ attention. The extra _heads_ dimension allows us to have multiple _representation subspaces_. It gives us 8 different ways of considering the same sentence.


<a href="https://www.lesswrong.com/posts/qscAeYE67GoSffDDA/walkthrough-the-transformer-architecture-part-1-2">Article about transformer</a>: Heart of the Transformer is the attention mechanism. The way majority of the people show attention is through a matrix with words. If the connected words, therefore the sentence, is the same in rows and columns, then we have the self-attention mechanism:

<img src='../img/transformer/self_attention.png' width="500" height="500">

With it we see which parts of the sentence are relevant to the other parts.
Also in case of CNN, for instance, we frequently describe neural networks as having some sort of structured model for the images we are classifying. This is why authors sometimes talk about the model recognizing smaller pieces of the picture, like wheels, and then combining these smaller features into a coherent whole, such as a car. 

The way I think of attention is similar. The hope is that the neural network will be able to capture those parts of the text that are related to each other.

## Decoder
The `Decoder` has 3 sublayers:
1. Masked multi head attention: mask future positions.
2. multi-head attention: encoder-decoder multi-head attention.
3. Feed-forward network: linear layer and softmax

__Masked multi-head attention__

We prevent positions from attending to subsequente positions.

__Encoder-Decoder multi-head attention__

`x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask`

`x` comes from the previous `DecoderLayer`, `m` comes from the `Encoder` output (`encoderLayer6`).
In the `Encoder-Decoder attention`  layers, ther queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence (because indeed the encoder has attend all the positions of the input sequence with mask only over the padded words).

### _Important notes_
We run the `Encoder` once to get the output of the encoder stack, which is the representation of the sentence "I like trees". Now we need to run the decoder multiple times so it can predict the words in Spanish "Me gustas los arboles".

The last layer expands the output of the decoder stack into a huge vector whose length is the `vocab-size`. The _softmax_ means that we'll select the one element of this huge vector with the highest probaility (_greedy decoding_).

During training the decoder might be not very good so it can produce incorrect predictions of the next word. If the decoder produces junk, we don't want to feed that junk back into the decoder for the next step. So, during training, we use a process called _teacher forcing_. With _teacher forcing_ we feed in the right translation till the previous point. The loss is then calculated using the probability distribution over possible next words that the decoder actually produced versus the probability distribution it should have produced, this means the loss is calculated based on the actual prediction of the decoder.

In [ ]:
class DecoderLayer(nn.Module):
    

In [15]:
a = torch.arange(0,10).reshape(2,5)

In [16]:
a.size()

torch.Size([2, 5])

In [17]:
a.unsqueeze(1).size()

torch.Size([2, 1, 5])

In [18]:
a.size(0)

2

In [8]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [10]:
d_model = 512
N = 6
linear = get_clones(nn.Linear(d_model, d_model), N); linear

ModuleList(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Linear(in_features=512, out_features=512, bias=True)
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): Linear(in_features=512, out_features=512, bias=True)
  (5): Linear(in_features=512, out_features=512, bias=True)
)

In [3]:
path = '../img/transformer/'

!ls {path}

encoder.png                  wordembedding_positional.png


In [4]:
torch.arange(0, 555).unsqueeze(-1).size()

torch.Size([555, 1])

In [5]:
torch.arange(0, 555).unsqueeze(1).size()

torch.Size([555, 1])

In [6]:
# !pip install torch

In [7]:
import math, copy, time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from typing import Type
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context='talk')

In [5]:
class Norm(nn.Module):
    """Residual connection around each of the two sub-layers, followed by layer normalization"""
    def __init__(self, features, eps=1e-6):
        super(Norm, self).__init__()
        self.a = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))
        self.eps = eps
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a * (x - mean) / (std + self.eps) + self.bias

In [6]:
x = np.array([2,6])

In [7]:
x

array([2, 6])

In [8]:
mean = x.mean(axis=-1, keepdims=True)

In [9]:
x - mean

array([-2.,  2.])

# Code
## _Attention_
Amazing explanation at: https://www.tensorflow.org/tutorials/text/transformer


Attention mechanism is the result of a few operations:

1. the dot product between Query $Q$ and Key $K$. 

$$Q \bullet K^T$$

2. It is then divided/scaled by the number of element $d$ in $K$ (or $Q$).

$$\frac{Q \bullet K^T}{\sqrt{d_k}}$$

3. After this we apply a softmax and multiply by $V$.

$$Attention(Q, K, V) = softmax(\frac{Q \bullet K^T}{\sqrt{d_k}}) \bullet V$$

In [10]:
def attention(query: Type[torch.Tensor], key: Type[torch.tensor],
              value: Type[torch.Tensor], mask=None, dropout=None):
    """Compute the scaled dot product attention"""
    
    d_k = query.size(-1)  # column size
    print(f'd_k value {d_k}')
    print(f'{query.size()} , {key.size()}, {value.size()}')
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    print(f'Score shape -- {scores.size()}')
    if mask is not None:
        print(f'Mask ---> {mask.size()}')
        mask = mask.unsqueeze(1)
        print(f'Mask new --> {mask.size()}')
        scores = scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim = -1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value)

In [11]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, heads, d_model = 512, dropout=0.1):
        """
        :params
        -------
        heads : int
            Number of heads. Heads = 8, in the original paper.
        d_model : int
            Number of sentences in the set. The `d_model` = 512 in the original paper 
            "Attention is all you need". 
        dropout : float
            Dropout value.
        
        :return
        -------
        output : torch.Tensor
            Tensor with attention matrix.
        """
        super(MultiHeadedAttention, self).__init__()
        
        assert d_model % heads == 0, 'There must be no remainder!'
        # Split the dimensionality into `h` heads. 512 / 8 = 64
        self.d_k = d_model // heads
        self.h = heads
        self.d_model = d_model
        
        # 4 layers: `Q`, `K`, `V`, `O`
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(d_model, d_model)
        
    def forward(self, q, k, v, mask):
        
        batch_size = q.size(0)  # number of rows in the `q` matrix
        
        # linear transformation and split into h heads
        q = self.q_linear(q).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
        k = self.k_linear(k).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
        v = self.v_linear(v).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
        
        # calculate attention
        # Attention tells you which words are important and to which we have to give more 
        # value: https://www.tensorflow.org/tutorials/text/transformer
        attention_weights = attention(q, k, v, mask=mask, dropout=self.dropout)
        
        # concatenate heads
        concat = attention_weights.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # The output represents the multiplication of the attention weights and the V (value)
        # vector. This ensures that the words you want to focus on are kept as-is and the
        # irrelevant words are flushed out.
        return self.output(concat)

## _Positional Encoding_

At the very beginning of the encoder.

In [12]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 200, dropout = 0.1):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        pe = Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        return self.dropout(x)

## _Masks_

We use masks in 2 cases:
1. in the encoder-decoder: to zero attention ouputs wherever there is just padding in the input sentences.
2. In the decoder: to prevent the decoder to peak ahead at the rest of the translated sentence when predicting next word.

### _Torchtext_

In [13]:
path = '../data/nlp/transformer/fr-en'
!ls {path}

europarl_en = open(f'{path}/europarl-v7.fr-en.en', encoding='utf-8').read().split('\n')
europarl_fr = open(f'{path}/europarl-v7.fr-en.fr', encoding='utf-8').read().split('\n')

#!python3 -m spacy download en
#!python3 -m spacy download fr
#!pip install torchtext==0.6.0
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import pandas as pd

en = spacy.load('en')
fr = spacy.load('fr')

def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]

def tokenize_fr(sentence):
    return [tok.text for tok in fr.tokenizer(sentence)]

# Before you create the field, and at then end you build the vocab from these two Fields
EN_TEXT = Field(tokenize=tokenize_en)
FR_TEXT = Field(tokenize=tokenize_fr, init_token='<sos>', eos_token='<eos>')

# csv format
raw_data = {'English' : [line for line in europarl_en[:5000]], 'French': [line for line in europarl_fr[:5000]]}

df = pd.DataFrame(raw_data, columns=["English", "French"])

# remove very long sentences and sentences where translations are 
# not of roughly equal length
df['eng_len'] = df['English'].str.count(' ')
df['fr_len'] = df['French'].str.count(' ')
df = df.query('fr_len < 80 & eng_len < 80')
df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')

# create train and validation set
train, val = train_test_split(df, test_size=0.1)

train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)

data_fields = [('English', EN_TEXT), ('French', FR_TEXT)]
train, val = TabularDataset.splits(path='./', train='train.csv', validation='val.csv', 
                                   format='csv', fields=data_fields)

FR_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)

train_iter = BucketIterator(train, batch_size=512, 
                            sort_key=lambda x: len(x.French), shuffle=True)

europarl-v7.fr-en.en europarl-v7.fr-en.fr


You can see we split in batches the sentences and from there `pytorch` creates the vocab size of your embeddings.

In [14]:
for i in train_iter:
    print(i)


[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 83x512]
	[.French]:[torch.LongTensor of size 87x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 87x512]
	[.French]:[torch.LongTensor of size 92x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 88x512]
	[.French]:[torch.LongTensor of size 92x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 92x512]
	[.French]:[torch.LongTensor of size 96x512]

[torchtext.data.batch.Batch of size 3]
	[.English]:[torch.LongTensor of size 37x3]
	[.French]:[torch.LongTensor of size 41x3]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 88x512]
	[.French]:[torch.LongTensor of size 92x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 89x512]
	[.French]:[torch.LongTensor of size 87x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of 

In [15]:
for i in iter(train_iter):
    print(i)


[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 83x512]
	[.French]:[torch.LongTensor of size 88x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 92x512]
	[.French]:[torch.LongTensor of size 96x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 89x512]
	[.French]:[torch.LongTensor of size 93x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 88x512]
	[.French]:[torch.LongTensor of size 95x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 87x512]
	[.French]:[torch.LongTensor of size 91x512]

[torchtext.data.batch.Batch of size 3]
	[.English]:[torch.LongTensor of size 25x3]
	[.French]:[torch.LongTensor of size 30x3]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of size 91x512]
	[.French]:[torch.LongTensor of size 88x512]

[torchtext.data.batch.Batch of size 512]
	[.English]:[torch.LongTensor of 

In [ ]:
for batch in iter(train_iter):
    print(batch.English.transpose(0, 1).size())

In [17]:
len(EN_TEXT.vocab.itos)

8181

In [18]:
len(FR_TEXT.vocab.itos)

10242

### _Encoder_

In [19]:
# ENCODER MASKS
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']
input_msk = (input_seq != input_pad).unsqueeze(1)

In [20]:
input_msk.shape

torch.Size([512, 1, 91])

### _Decoder_

In [23]:
# DECODER MASKS
target_seq = batch.French.transpose(0,1)
target_pad = FR_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) # get seq_len for matrix
nopeak_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
target_msk = target_msk & nopeak_mask

In [24]:
target_msk.shape

torch.Size([512, 91, 91])

## _Embedder_
The `Embedder` has as parameters `vocab_size` and `d_model`. The `vocab_size` is defined by the number of unique words the encoded text has. In this case you can use `len(EN_TEXT.vocab.itos)` to understand what this parameter will be. The same will happen also for the French language in the decoder. The `d_model` is set to 512 in the paper and here too.

In [25]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

## _Feed-Forward_

In [26]:
class FeedForward(nn.Module):
    """Linear transformation and a ReLU"""
    def __init__(self, d_model: int = 512, d_ff: int = 2048, dropout: float = 0.1):
        super(FeedForward, self).__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        return self.linear_2(x)

## Layers _Encoder - Decoder_

In [27]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadedAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        print(f'this is x in EncoderLayer-----> {x}')
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2, x2, x2, mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
        

class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadedAttention(heads, d_model)
        self.attn_2 = MultiHeadedAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        
    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x

In [28]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [29]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        
        x = self.pe(x)
        print(f'--Encoder x after PE : {x}')
        for i in range(self.N):
            print(f'loop within encoder. Iteration -- {i}')
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(DecoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

## _Transformer_

In [30]:
class Transformer(nn.Module):
    """Transformer architecture with encoder, decoder and Linear layer of the output."""
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads, dropout = 0.1):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, dropout)
        self.decoder = Decoder(trg_vocab, d_model, N, heads, dropout)
        self.out = nn.Linear(d_model, trg_vocab)
    
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

## _Model_

In [31]:
def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    return np_mask

def create_masks(src, trg):
    
    source_pad = EN_TEXT.vocab.stoi['<pad>']
    src_mask = (src != source_pad).unsqueeze(-2)

    if trg is not None:
        target_pad = FR_TEXT.vocab.stoi['<pad>']
        trg_mask = (trg != target_pad).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [41]:
d_model = 512
heads = 8
N = 6
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(FR_TEXT.vocab)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads, dropout = 0.1)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

def train_model(epochs, print_every=100):
    
    model.train()
    
    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
        print(f'Epoch - {epoch}')
        for i, batch in enumerate(train_iter):
            src = batch.English.transpose(0,1)
            trg = batch.French.transpose(0,1)
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            src_mask, trg_mask = create_masks(src, trg_input)
            
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            print(f'predictions size : {preds.view(-1, preds.size(-1)).size()}')
            print(f'targets size : {targets.size()}')
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), targets, 
                                   ignore_index=target_pad)
            loss.backward()
            optim.step()
            print(f'Loss.data ----> = {loss.item()}')
            total_loss += loss.item()
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg, time.time() - temp, print_every))
                total_loss = 0
                temp = time.time()

In [42]:
train_model(epochs=3)

Epoch - 0
--Encoder x after PE : tensor([[[-0.0851,  0.9312, -0.2281,  ...,  1.0702, -0.1880,  1.0272],
         [ 0.4032,  0.4439,  1.3056,  ...,  0.5143, -0.6426,  0.7896],
         [ 1.6677, -0.8952,  1.6084,  ...,  0.0000, -0.2544,  0.0000],
         ...,
         [-0.2319,  0.5529, -0.4102,  ...,  1.0595,  0.0000,  0.5406],
         [-0.8427, -0.4381, -0.2582,  ...,  1.0595,  0.0000,  0.0000],
         [-0.4383, -1.2356,  0.0000,  ...,  1.0595,  0.1775,  0.0000]],

        [[ 0.3753,  0.5082, -0.4723,  ...,  1.5692, -0.3355,  0.8619],
         [ 0.6853,  0.2364,  0.0000,  ...,  0.0000, -0.6110,  0.5476],
         [ 1.4878, -0.0000,  1.2968,  ...,  1.5232,  0.5940,  1.3578],
         ...,
         [-0.2319,  0.0000, -0.4102,  ...,  1.0595,  0.1775,  0.5406],
         [-0.8427, -0.4381, -0.2582,  ...,  1.0595,  0.0000,  0.5406],
         [-0.4383, -1.2356,  0.6229,  ...,  1.0595,  0.1775,  0.5406]],

        [[-0.5318,  0.9221,  0.4146,  ...,  0.5143, -0.6426,  0.7896],
         [ 1

d_k value 64
torch.Size([512, 8, 82, 64]) , torch.Size([512, 8, 82, 64]), torch.Size([512, 8, 82, 64])
Score shape -- torch.Size([512, 8, 82, 82])
Mask ---> torch.Size([512, 1, 82])
Mask new --> torch.Size([512, 1, 1, 82])
loop within encoder. Iteration -- 2
this is x in EncoderLayer-----> tensor([[[ 1.2373,  2.3735,  1.1324,  ..., -2.2191,  0.6009, -0.5536],
         [ 1.7796,  0.2496,  0.9270,  ..., -1.8907, -0.7796, -0.0823],
         [ 3.6477, -1.6047,  2.9605,  ..., -2.2155,  1.5546, -1.9528],
         ...,
         [ 1.7254,  1.0542, -0.1385,  ..., -0.8589,  1.2703, -1.6588],
         [ 0.2856, -0.4140, -0.2438,  ..., -0.9472,  1.7512, -1.1925],
         [ 0.2425,  0.7415, -0.1423,  ..., -2.3492,  1.3482, -1.8761]],

        [[ 1.0044,  0.9614,  0.1505,  ..., -0.2430,  2.1106, -1.6201],
         [ 0.8209,  0.7809,  0.1609,  ..., -2.1277,  1.1611, -2.1375],
         [ 2.6876, -0.1629,  3.0896,  ..., -0.2157,  3.3580,  0.1907],
         ...,
         [ 0.5951,  1.6866,  1.0686,  ..

d_k value 64
torch.Size([512, 8, 82, 64]) , torch.Size([512, 8, 82, 64]), torch.Size([512, 8, 82, 64])
Score shape -- torch.Size([512, 8, 82, 82])
Mask ---> torch.Size([512, 1, 82])
Mask new --> torch.Size([512, 1, 1, 82])
loop within encoder. Iteration -- 5
this is x in EncoderLayer-----> tensor([[[ 2.3736, -1.0162,  0.3866,  ..., -1.9666, -1.3029, -0.2442],
         [ 1.9583, -3.2455, -1.9080,  ..., -0.1589, -3.6666,  0.8351],
         [ 3.8433, -2.2394,  3.1165,  ..., -0.8794, -0.7147, -0.0499],
         ...,
         [ 1.3359, -0.6574, -0.4339,  ...,  0.7388, -0.1017, -2.0900],
         [ 1.1404, -2.7856, -0.8308,  ..., -0.8665, -1.4210, -0.2405],
         [ 0.6618, -2.0113, -1.9549,  ..., -0.8180, -1.5113, -0.4708]],

        [[ 0.9547, -1.3453, -1.3675,  ..., -0.8117, -0.0519, -0.9876],
         [ 2.4575, -3.6740,  0.1999,  ..., -2.2720, -2.2707, -1.7620],
         [ 3.5713, -3.5491,  4.1238,  ..., -0.1193, -0.2347, -0.6394],
         ...,
         [ 2.5983, -2.4574,  1.8107,  ..

d_k value 64
torch.Size([512, 8, 92, 64]) , torch.Size([512, 8, 92, 64]), torch.Size([512, 8, 92, 64])
Score shape -- torch.Size([512, 8, 92, 92])
Mask ---> torch.Size([512, 1, 92])
Mask new --> torch.Size([512, 1, 1, 92])
loop within encoder. Iteration -- 1
this is x in EncoderLayer-----> tensor([[[-2.5324,  3.7967,  0.0216,  ...,  0.1651,  1.2800,  0.3328],
         [-0.2523,  2.2308,  1.2020,  ..., -0.2065,  1.0067,  0.0549],
         [ 1.6817, -0.0502,  0.7137,  ...,  0.4918,  1.3011,  0.5103],
         ...,
         [-0.0726,  0.3920,  2.4521,  ..., -0.1285,  1.1017, -0.8300],
         [-0.0763,  2.1565,  2.3440,  ..., -0.8168,  0.0890, -0.9893],
         [-0.9927,  2.3537,  0.8408,  ..., -0.2144,  1.4347, -0.3247]],

        [[-0.8982,  2.7631,  1.0055,  ..., -0.0910,  0.8583,  0.5200],
         [ 0.1012,  1.8462,  1.1131,  ..., -1.1167,  0.2531,  0.8317],
         [-0.3562,  1.0680,  1.5284,  ..., -0.5806,  1.5817,  1.5641],
         ...,
         [-0.0596,  1.1810,  2.2705,  ..

d_k value 64
torch.Size([512, 8, 92, 64]) , torch.Size([512, 8, 92, 64]), torch.Size([512, 8, 92, 64])
Score shape -- torch.Size([512, 8, 92, 92])
Mask ---> torch.Size([512, 1, 92])
Mask new --> torch.Size([512, 1, 1, 92])
loop within encoder. Iteration -- 4
this is x in EncoderLayer-----> tensor([[[-1.1884,  5.1994,  0.1242,  ..., -3.6180, -1.2333, -2.5598],
         [ 1.5916,  2.8187,  1.3063,  ..., -4.1959, -1.9222, -3.3988],
         [ 3.3879,  3.3241, -0.2021,  ..., -3.9558, -0.1180, -3.2054],
         ...,
         [ 1.3802,  1.4840,  0.7675,  ..., -3.8833, -1.3094, -2.0298],
         [ 0.4350,  2.9146,  2.1709,  ..., -4.7154, -3.6078, -2.8101],
         [-0.1253,  4.4905, -1.2041,  ..., -4.4684, -0.4421, -0.9889]],

        [[-0.2393,  4.2441, -0.2232,  ..., -4.4183, -0.4282, -2.1799],
         [ 1.0243,  3.6881, -0.1542,  ..., -5.6110, -1.0574, -2.0070],
         [ 1.0418,  1.5724,  0.4145,  ..., -3.3089,  0.1020, -1.5662],
         ...,
         [ 0.6121,  2.1321,  0.8328,  ..

...

Score shape -- torch.Size([3, 8, 47, 47])
Mask ---> torch.Size([3, 47, 47])
Mask new --> torch.Size([3, 1, 47, 47])
d_k value 64
torch.Size([3, 8, 47, 64]) , torch.Size([3, 8, 38, 64]), torch.Size([3, 8, 38, 64])
Score shape -- torch.Size([3, 8, 47, 38])
Mask ---> torch.Size([3, 1, 38])
Mask new --> torch.Size([3, 1, 1, 38])
d_k value 64
torch.Size([3, 8, 47, 64]) , torch.Size([3, 8, 47, 64]), torch.Size([3, 8, 47, 64])
Score shape -- torch.Size([3, 8, 47, 47])
Mask ---> torch.Size([3, 47, 47])
Mask new --> torch.Size([3, 1, 47, 47])
d_k value 64
torch.Size([3, 8, 47, 64]) , torch.Size([3, 8, 38, 64]), torch.Size([3, 8, 38, 64])
Score shape -- torch.Size([3, 8, 47, 38])
Mask ---> torch.Size([3, 1, 38])
Mask new --> torch.Size([3, 1, 1, 38])
d_k value 64
torch.Size([3, 8, 47, 64]) , torch.Size([3, 8, 47, 64]), torch.Size([3, 8, 47, 64])
Score shape -- torch.Size([3, 8, 47, 47])
Mask ---> torch.Size([3, 47, 47])
Mask new --> torch.Size([3, 1, 47, 47])
d_k value 64
torch.Size([3, 8, 47, 6

d_k value 64
torch.Size([512, 8, 83, 64]) , torch.Size([512, 8, 83, 64]), torch.Size([512, 8, 83, 64])
Score shape -- torch.Size([512, 8, 83, 83])
Mask ---> torch.Size([512, 1, 83])
Mask new --> torch.Size([512, 1, 1, 83])
loop within encoder. Iteration -- 1
this is x in EncoderLayer-----> tensor([[[-3.2083,  2.7163, -0.4514,  ..., -1.0945,  2.0085, -0.7440],
         [-1.2089,  3.4290, -0.2336,  ...,  0.2839,  3.2408, -0.1589],
         [-1.2459, -0.0802, -0.2212,  ..., -0.1571,  3.2763, -1.8929],
         ...,
         [-2.6418,  3.2105, -1.0993,  ..., -1.9193,  1.5238, -0.5554],
         [-2.4292,  2.1361,  1.5128,  ..., -0.0404,  2.7486, -1.6413],
         [-0.9099,  1.7086,  0.5694,  ..., -2.0088,  2.3590, -1.0589]],

        [[-1.7956,  4.3440, -0.3535,  ..., -0.6036,  2.4987,  0.8666],
         [-1.6941,  3.1758,  0.5202,  ..., -1.4637,  2.5755, -0.5400],
         [-1.1819,  2.0917,  0.9725,  ..., -1.1468,  3.0812,  0.3517],
         ...,
         [-1.9340,  3.0544, -0.8645,  ..

d_k value 64
torch.Size([512, 8, 83, 64]) , torch.Size([512, 8, 83, 64]), torch.Size([512, 8, 83, 64])
Score shape -- torch.Size([512, 8, 83, 83])
Mask ---> torch.Size([512, 1, 83])
Mask new --> torch.Size([512, 1, 1, 83])
loop within encoder. Iteration -- 4
this is x in EncoderLayer-----> tensor([[[-4.1193,  6.6429, -3.8450,  ..., -7.2528, -1.2976, -6.2127],
         [-2.1995,  9.5656, -3.7744,  ..., -3.4927, -1.5488, -5.9292],
         [ 0.1318,  5.3354, -2.0962,  ..., -6.0749,  1.8140, -7.6801],
         ...,
         [-2.5728, 10.1480, -2.2318,  ..., -6.7148, -3.0298, -6.0873],
         [-1.9343,  8.2127, -1.1278,  ..., -5.7940, -0.7903, -6.3595],
         [-1.2767,  7.9044, -2.0837,  ..., -8.1152, -2.0378, -6.9550]],

        [[ 0.4073,  9.8701, -2.8329,  ..., -5.2840, -2.0701, -5.7113],
         [-0.8818,  7.9978, -1.6759,  ..., -7.1254, -0.9227, -7.2638],
         [ 0.5060,  8.1224, -1.2687,  ..., -6.8281, -0.8565, -5.6385],
         ...,
         [-0.0898,  9.0096, -4.4428,  ..

d_k value 64
torch.Size([512, 8, 87, 64]) , torch.Size([512, 8, 87, 64]), torch.Size([512, 8, 87, 64])
Score shape -- torch.Size([512, 8, 87, 87])
Mask ---> torch.Size([512, 87, 87])
Mask new --> torch.Size([512, 1, 87, 87])
d_k value 64
torch.Size([512, 8, 87, 64]) , torch.Size([512, 8, 83, 64]), torch.Size([512, 8, 83, 64])
Score shape -- torch.Size([512, 8, 87, 83])
Mask ---> torch.Size([512, 1, 83])
Mask new --> torch.Size([512, 1, 1, 83])
d_k value 64
torch.Size([512, 8, 87, 64]) , torch.Size([512, 8, 87, 64]), torch.Size([512, 8, 87, 64])
Score shape -- torch.Size([512, 8, 87, 87])
Mask ---> torch.Size([512, 87, 87])
Mask new --> torch.Size([512, 1, 87, 87])
d_k value 64
torch.Size([512, 8, 87, 64]) , torch.Size([512, 8, 83, 64]), torch.Size([512, 8, 83, 64])
Score shape -- torch.Size([512, 8, 87, 83])
Mask ---> torch.Size([512, 1, 83])
Mask new --> torch.Size([512, 1, 1, 83])
predictions size : tensor([[-0.9196, -1.0088, -0.5155,  ..., -0.6532, -0.7871, -0.2975],
        [-0.978

KeyboardInterrupt: 

In [ ]:
matrix = torch.randn(1,16).reshape(4,4)

In [ ]:
matrix.dim()

In [ ]:
matrix.size()

In [ ]:
matrix

In [ ]:
matrix_2 = matrix.unsqueeze(-2)

In [ ]:
matrix_2.size()  # 4 batches, 1 row, 4 columns

In [ ]:
matrix_2

In [ ]:
matrix_3 = matrix.unsqueeze(-1); matrix_3.size()

In [ ]:
matrix_3  # 4 batches, 4 rows, 1 column

In [ ]:
matrix.unsqueeze(-3).size()